# SA405 Lesson 8

### Today...

- (I) Read and write to Excel using the Python library pandas, xlrd, xlwt, and XLsxWriter
- (II) Test for multiple optimal solutions to an LP/IP
- (III) Homework is at the end!

## (I) Read and write to Excel using xlwings

#### (A) See if you can import xlwings by running the cell below.  
1. If not, you need to install pandas:
   -  in an anaconda prompt, type `conda install -c conda-forge pandas`
2. If so, it's not a bad idea to update the installation of xlwings
   -  in an anaconda prompt, type `conda update -c conda-forge pandas`

In [4]:
conda install -c conda-forge xlrd

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py37hf985489_0         3.0 MB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/osx-64::python_abi-3.7-2_cp37m

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.10.3-py37hf985489_0



p

In [5]:
conda install -c conda-forge xlwt

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
conda install -c conda-forge xlsxwriter

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import xlwt
import xlsxwriter

### Python function to build model

#### (B) Check out the code and make note of any questions.

In [8]:
def sudoku_model(puzzle_values):
    """
    Build an IP to solve a 9x9 Sudoku puzzle
    
    Keyword arguments:
    puzzle_values -- prefilled puzzle values as a list of 3-tuples (row,col,num)
    
    Return:
    Pyomo model
    """
    # Sets
    NUMS = [1,2,3,4,5,6,7,8,9]
    BOX = [1,2,3]
    BOX_0 = [0,1,2]
    
    model = pyo.ConcreteModel()

    #Decision variables
    model.x = pyo.Var(NUMS,NUMS,NUMS,domain=pyo.Binary)

    #Objective function
    def obj_rule(model):
        return sum(model.x[row,col,num] for row in NUMS for col in NUMS for num in NUMS)
    model.obj = pyo.Objective(rule=obj_rule,sense=pyo.minimize)

    #No duplicates in any row
    def row_rule(model,row,num):
        return sum(model.x[row,col,num] for col in NUMS) == 1
    model.row_constraint = pyo.Constraint(NUMS,NUMS,rule=row_rule)

    #No duplicates in any column
    def col_rule(model,col,num):
        return sum(model.x[row,col,num] for row in NUMS) == 1
    model.col_constraint = pyo.Constraint(NUMS,NUMS,rule=col_rule)

    #No duplicates in any major 3x3 box
    def three_by_three_rule(model,box_row,box_col,num):
        return sum(model.x[3*box_row + row_pos,3*box_col + col_pos,num] for row_pos in BOX for col_pos in BOX) == 1
    model.three_by_three_constraint = pyo.Constraint(BOX_0,BOX_0,NUMS,rule=three_by_three_rule)

    #Only one number per small box
    def box_rule(model,row,col):
        return sum(model.x[row,col,num] for num in NUMS) == 1
    model.box_constraint = pyo.Constraint(NUMS,NUMS,rule=box_rule)

    #Prefilled values
    def prefilled_rule(model,row,col,num):
        return model.x[row,col,num] == 1
    model.prefilled_constraint = pyo.Constraint(puzzle_values,rule=prefilled_rule)
    
    return model

### Read puzzle values from Excel and build PREFILLED list

- We will build the following PREFILLED list by reading the values in from Excel

In [9]:
# PREFILLED = [(1,1,3),(1,6,5),(1,9,9),
#              (2,2,9),(2,8,2),
#              (3,3,4),(3,5,8),(3,8,7),
#              (4,5,2),(4,6,7),(4,7,9),(4,8,1),
#              (5,3,9),(5,5,1),(5,7,4),
#              (6,2,1),(6,3,8),(6,4,5),(6,5,4),
#              (7,2,4),(7,5,5),(7,7,8),
#              (8,2,2),(8,8,9),
#              (9,1,8),(9,4,6),(9,9,2)]

#### (C) We will use `range()` to index over lists.  Run the cell below.
1. How many values does `range(9)` have? 
   - *Answer: 9*
2. What number does it start with?  
   - *Answer: 0*

(Recall that list indexing in Python begins at 0.)

In [ ]:
list(range(9))

#### (D) Open workbook, open worksheet, read data.
1. Open the workbook called "sudoku_puzzle.xlsx" as `wb` (to make this easier, make sure the workbook is in the same directory as this notebook)
2. Open the sheet in `wb` called "Puzzle"as `puzzle_sheet`
3. Read the values in range "A1:I9" in `puzzle_sheet` into `values`
4. Print `values` to see the format of what was read in from Excel

In [40]:
# open the workbook
wb = pd.read_excel('sudoku_puzzle.xlsx')


# open the sheet with the puzzle
puzzle_sheet = pd.read_excel('sudoku_puzzle.xlsx', sheet_name= 'Puzzle')


# read in the puzzle
values = puzzle_sheet

# print(values.iloc[0,0])
print(values)

3.0


- See all those `NaN`s?  We need to bypass those and record an entry in PREFILLED for every numeric value in `values`.

#### (E) Make PREFILLED list: Index over the positions in `values` using a double for loop.  Within the loop:
1. Save the value at the current position as `num`
2. If `num` is a float: `if type(num)==float`,
3. append the appropriate (row,col,num) tuple to the list `PREFILLED` 
(Remember that we are indexing starting at 1 in our code)
4. Instead of using `num` directly, cast `num` as an integer: `int(num)`

- Print `PREFILLED`.  It should be the same list of tuples as above:  `[(1,1,3),(1,6,5),(1,9,9),...`

In [64]:
# Build the list of prefilled values using a double for loop
PREFILLED = []
for m in range(9):
    for n in range(9):
        num = values.iloc[m,n]
        if num > 0:
            PREFILLED.append((m+1,n+1,int(num)))
        
        
# print(num)
print(f'Using double for loop: {PREFILLED}')

# OR #

# we can put the same logic in a list comprehension (a more Python-y solution)   
# PREFILLED = # Here

print(f'Using list comprehension: {PREFILLED}')

Using double for loop: [(1, 1, 3), (1, 6, 5), (1, 9, 9), (2, 2, 9), (2, 8, 2), (3, 3, 4), (3, 5, 8), (3, 8, 7), (4, 5, 2), (4, 6, 7), (4, 7, 9), (4, 8, 1), (5, 3, 9), (5, 5, 1), (5, 7, 4), (6, 2, 1), (6, 3, 8), (6, 4, 5), (6, 5, 4), (7, 2, 4), (7, 5, 5), (7, 7, 8), (8, 2, 2), (8, 8, 9), (9, 1, 8), (9, 4, 6)]
Using list comprehension: [(1, 1, 3), (1, 6, 5), (1, 9, 9), (2, 2, 9), (2, 8, 2), (3, 3, 4), (3, 5, 8), (3, 8, 7), (4, 5, 2), (4, 6, 7), (4, 7, 9), (4, 8, 1), (5, 3, 9), (5, 5, 1), (5, 7, 4), (6, 2, 1), (6, 3, 8), (6, 4, 5), (6, 5, 4), (7, 2, 4), (7, 5, 5), (7, 7, 8), (8, 2, 2), (8, 8, 9), (9, 1, 8), (9, 4, 6)]


### Run the solver

In [65]:
# Call function to build model, sending PREFILLED as argument
model = sudoku_model(puzzle_values=PREFILLED)

# Solve the model
solver_result = pyo.SolverFactory('glpk').solve(model)

### Print solution here in the Jupyter notebook

- This function prints the solution here in the notebook

In [66]:
def print_sol_here(model):
    # Check if the model solved to optimality before printing solution
    solve_status = solver_result.solver.termination_condition
    if (solve_status==pyo.TerminationCondition.optimal):
        NUMS = [1,2,3,4,5,6,7,8,9]
        for row in NUMS:
            for col in NUMS:
                for num in NUMS:
                    if model.x[row,col,num] == 1:
                        print(f'({row},{col}) = {num}')
    else:
        print(f'The solver status is {solve_status}')

In [67]:
print_sol_here(model)

(1,1) = 3
(1,2) = 8
(1,3) = 2
(1,4) = 7
(1,5) = 6
(1,6) = 5
(1,7) = 1
(1,8) = 4
(1,9) = 9
(2,1) = 7
(2,2) = 9
(2,3) = 5
(2,4) = 4
(2,5) = 3
(2,6) = 1
(2,7) = 6
(2,8) = 2
(2,9) = 8
(3,1) = 1
(3,2) = 6
(3,3) = 4
(3,4) = 9
(3,5) = 8
(3,6) = 2
(3,7) = 5
(3,8) = 7
(3,9) = 3
(4,1) = 4
(4,2) = 5
(4,3) = 3
(4,4) = 8
(4,5) = 2
(4,6) = 7
(4,7) = 9
(4,8) = 1
(4,9) = 6
(5,1) = 2
(5,2) = 7
(5,3) = 9
(5,4) = 3
(5,5) = 1
(5,6) = 6
(5,7) = 4
(5,8) = 8
(5,9) = 5
(6,1) = 6
(6,2) = 1
(6,3) = 8
(6,4) = 5
(6,5) = 4
(6,6) = 9
(6,7) = 7
(6,8) = 3
(6,9) = 2
(7,1) = 9
(7,2) = 4
(7,3) = 7
(7,4) = 2
(7,5) = 5
(7,6) = 3
(7,7) = 8
(7,8) = 6
(7,9) = 1
(8,1) = 5
(8,2) = 2
(8,3) = 6
(8,4) = 1
(8,5) = 7
(8,6) = 8
(8,7) = 3
(8,8) = 9
(8,9) = 4
(9,1) = 8
(9,2) = 3
(9,3) = 1
(9,4) = 6
(9,5) = 9
(9,6) = 4
(9,7) = 2
(9,8) = 5
(9,9) = 7


### Print solution to Excel

- We want to print the solution to a sheet in the Excel workbook `wb`
- First we need to put the solution into a 9x9 grid like `values` above: a list of 9 lists (rows), each with 9 values.

#### (F) Loop over the rows, columns, and numbers to put solution into 9x9 grid.  
-  If the $x$ value is 1, put the number into `solution[row-1][col-1]`

In [76]:
# Build a 9x9 grid called "solution" to hold the solution
solution = {}
print(f'Before:{solution}')    

# Use this list for indexing the for loops
NUMS = [1,2,3,4,5,6,7,8,9]


# Set the values to the solution found by the solver
for row in NUMS:
    for col in NUMS:
        for num in NUMS:
            if model.x[row,col,num] == 1:
                solution[row,col] = num
        
print(f'After:{solution}')    

Before:{}
After:{(1, 1): 3, (1, 2): 8, (1, 3): 2, (1, 4): 7, (1, 5): 6, (1, 6): 5, (1, 7): 1, (1, 8): 4, (1, 9): 9, (2, 1): 7, (2, 2): 9, (2, 3): 5, (2, 4): 4, (2, 5): 3, (2, 6): 1, (2, 7): 6, (2, 8): 2, (2, 9): 8, (3, 1): 1, (3, 2): 6, (3, 3): 4, (3, 4): 9, (3, 5): 8, (3, 6): 2, (3, 7): 5, (3, 8): 7, (3, 9): 3, (4, 1): 4, (4, 2): 5, (4, 3): 3, (4, 4): 8, (4, 5): 2, (4, 6): 7, (4, 7): 9, (4, 8): 1, (4, 9): 6, (5, 1): 2, (5, 2): 7, (5, 3): 9, (5, 4): 3, (5, 5): 1, (5, 6): 6, (5, 7): 4, (5, 8): 8, (5, 9): 5, (6, 1): 6, (6, 2): 1, (6, 3): 8, (6, 4): 5, (6, 5): 4, (6, 6): 9, (6, 7): 7, (6, 8): 3, (6, 9): 2, (7, 1): 9, (7, 2): 4, (7, 3): 7, (7, 4): 2, (7, 5): 5, (7, 6): 3, (7, 7): 8, (7, 8): 6, (7, 9): 1, (8, 1): 5, (8, 2): 2, (8, 3): 6, (8, 4): 1, (8, 5): 7, (8, 6): 8, (8, 7): 3, (8, 8): 9, (8, 9): 4, (9, 1): 8, (9, 2): 3, (9, 3): 1, (9, 4): 6, (9, 5): 9, (9, 6): 4, (9, 7): 2, (9, 8): 5, (9, 9): 7}


In [95]:
# Now print the solution in the Excel spreadsheet into a new sheet called "Solution1"
# try:
book = xlwt.Workbook()
sheet1 = book.add_sheet("SolutionSheet")


for row in NUMS:
    for col in NUMS:
        sheet1.write(col-1,row-1,solution[row,col])
        
book.save("test.xls")

In [98]:
# If you want to clear the values and/or delete the sheet
# sheet1.clear()
# sheet1.delete()

#### (G) Put code to print to Excel into a Python function.  (Check solver status before printing.) Function parameters:
-  model -- to access the solution
-  wb -- workbook to write solution in
-  sheet_name -- (string) name for the added sheet

In [125]:
def print_sol_excel(model, wb, sheet_name):
    """
    Print sudoku solution to Excel in a sheet called "sheet_name" (add the sheet if it doesn't already exist)
    
    Keyword Arguments:
    model -- Pyomo Sudoku model that has already been solved
    wb -- Excel Workbook
    sheet_name -- (string) name of the sheet where the solution is to be printed
    
    Return:
    nothing
    """
    # Check if the model solved to optimality before printing solution
    solve_status = solver_result.solver.termination_condition
    if (solve_status==pyo.TerminationCondition.optimal):
        # Code to print to Excel here
        # Use this list for indexing the for loops
        NUMS = [1,2,3,4,5,6,7,8,9]


        # Set the values to the solution found by the solver
        for row in NUMS:
            for col in NUMS:
                for num in NUMS:
                    if model.x[row,col,num] == 1:
                        solution[row,col] = num
        
        book = xlwt.Workbook()
        sheet1 = book.add_sheet("SolutionSheet")


        for row in NUMS:
            for col in NUMS:
                sheet1.write(col,row,solution[row,col])

        book.save(sheet_name)
        
        
        print(f'The solution was printed to Excel')
    else:
        print(f'The solver status is {solve_status}')

In [126]:
print_sol_excel(model, wb, "Solution_1")

The solution was printed to Excel


## (II) Test for another optimal solution

Sometimes we will want to explore multiple optimal solutions to a model. One way to check for another solution is to: 
1. Add a constraint to the model that removes the current optimal solution from the feasible region
   - The new constraint varies by problem.  It may take some creativity.
2. Rerun the model with the new constraint and see what happens

In this case, the following new constraint works:
$$ \sum_{i,j,k \in N:x_{i,j,k}=1} x_{i,j,k} \leq 80 $$
- Notice that the 81 $x$-values that have the value $1$ in the current solution are summed on the left
- This forces at least one of the $x$-values that was $1$ in the current solution to be $0$ in the new solution

#### (H) Add the above constraint to the model.

In [134]:
# Add the constraint defined above to the model
def exclude_sol_rule(model):
    return sum(model.x[row,col,num] for row in NUMS for col in NUMS for num in NUMS if model.x[row,col,num]==1) <= 80
#     return sum(model.x[i,j,k] for i in NUMS for j in NUMS for k in NUMS if model.x[i,j,k]) <= 80
model.exclude_sol_const = pyo.Constraint(rule=exclude_sol_rule)

    (type=<class 'pyomo.core.base.constraint.SimpleConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [135]:
# Print the new constraint, just to see if it looks like we expect
print(model.exclude_sol_const.expr)

x[1,1,3] + x[1,2,8] + x[1,3,2] + x[1,4,7] + x[1,5,6] + x[1,6,5] + x[1,7,1] + x[1,8,4] + x[1,9,9] + x[2,1,7] + x[2,2,9] + x[2,3,5] + x[2,4,4] + x[2,5,3] + x[2,6,1] + x[2,7,6] + x[2,8,2] + x[2,9,8] + x[3,1,1] + x[3,2,6] + x[3,3,4] + x[3,4,9] + x[3,5,8] + x[3,6,2] + x[3,7,5] + x[3,8,7] + x[3,9,3] + x[4,1,4] + x[4,2,5] + x[4,3,3] + x[4,4,8] + x[4,5,2] + x[4,6,7] + x[4,7,9] + x[4,8,1] + x[4,9,6] + x[5,1,2] + x[5,2,7] + x[5,3,9] + x[5,4,3] + x[5,5,1] + x[5,6,6] + x[5,7,4] + x[5,8,8] + x[5,9,5] + x[6,1,6] + x[6,2,1] + x[6,3,8] + x[6,4,5] + x[6,5,4] + x[6,6,9] + x[6,7,7] + x[6,8,3] + x[6,9,2] + x[7,1,9] + x[7,2,4] + x[7,3,1] + x[7,4,2] + x[7,5,5] + x[7,6,3] + x[7,7,8] + x[7,8,6] + x[7,9,7] + x[8,1,5] + x[8,2,2] + x[8,3,6] + x[8,4,1] + x[8,5,7] + x[8,6,8] + x[8,7,3] + x[8,8,9] + x[8,9,4] + x[9,1,8] + x[9,2,3] + x[9,3,7] + x[9,4,6] + x[9,5,9] + x[9,6,4] + x[9,7,2] + x[9,8,5] + x[9,9,1]  <=  80.0


#### (I) Run the model again and print the solution to a sheet called 'Solution2'.
1.  What does it mean if the model with the new constraint is infeasible?  
   - *Answer: there was only one solution to the puzzle!*
2.  Now remove a number from the puzzle in Excel (remember to save changes!) and run everything again from the top.  Is there another solution now?  
   - *Answer: Yes, now there is enough flexibility for multiple solutions to the puzzle.*

In [136]:
# Solve the model again
solver_result1 = pyo.SolverFactory('glpk').solve(model)

# Try to print the solution
print_sol_excel(model, wb, 'Solution2')

The solution was printed to Excel


# Lesson 8 HOMEWORK

### (HW1) Now add a second constraint to the model to exclude the second solution.  
-  Hint:  You need to give it a different name!  Maybe "exclude_sol2_const".

In [144]:
# Add the constraint defined above to the model

def exclude_sol2_rule(model):
    return sum(model.x[row,col,num] for row in NUMS for col in NUMS for num in NUMS if model.x[row,col,num]==1) <= 80
#     return sum(model.x[i,j,k] for i in NUMS for j in NUMS for k in NUMS if model.x[i,j,k]) <= 80
model.exclude_sol2_const = pyo.Constraint(rule=exclude_sol2_rule)


    (type=<class 'pyomo.core.base.constraint.SimpleConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


### (HW2) Solve the model again to see if a third solution exists.  If so, print the third solution in a different sheet.
-  Hint:  You can use the print function that's already written with a different sheet name.

In [150]:
# Solve the model again
solver_result2 = pyo.SolverFactory('glpk').solve(model)

# Try to print the solution
print_sol_excel(model, wb, 'Solution3')

The solution was printed to Excel


### (HW3) Read in the first and second solutions from the sheets in the workbook and count how many numbers they differ by.  Write the answer to this question on another sheet called "Difference"  
- Each solution will need its own variable name.  
- Use nested for loops along with a "counter" variable initialized a 0 to count how many squares the solutions differ by. 

In [164]:
counter = 0

wb2 = pd.read_excel('Solution2')
wb3 = pd.read_excel('Solution3')


# open the sheet with the puzzle
solution_sheet2 = pd.read_excel('Solution1', sheet_name= 'SolutionSheet')
solution_sheet3 = pd.read_excel('Solution3', sheet_name= 'SolutionSheet')


# read in the puzzle
values2 = solution_sheet2
values3 = solution_sheet3

# print(values.iloc[0,0])
# print(values)
for row in NUMS:
    for col in NUMS:
        print(values2.iloc[row-1,col])
        print(values3.iloc[row-1,col])
        if values2.iloc[row-1,col] == values3.iloc[row-1,col]:
            print("same value")
        else:
            counter +=1
print("Differ by ", counter, " numbers")
        

3
3
same value
7
7
same value
1
1
same value
4
4
same value
2
2
same value
6
6
same value
9
9
same value
5
5
same value
8
8
same value
8
8
same value
9
9
same value
6
6
same value
5
5
same value
7
7
same value
1
1
same value
4
4
same value
2
2
same value
3
3
same value
2
2
same value
5
5
same value
4
4
same value
3
3
same value
9
9
same value
8
8
same value
7
7
same value
6
6
same value
1
1
same value
7
7
same value
4
4
same value
9
9
same value
8
8
same value
3
3
same value
5
5
same value
2
2
same value
1
1
same value
6
6
same value
6
6
same value
3
3
same value
8
8
same value
2
2
same value
1
1
same value
4
4
same value
5
5
same value
7
7
same value
9
9
same value
5
5
same value
1
1
same value
2
2
same value
7
7
same value
6
6
same value
9
9
same value
3
3
same value
8
8
same value
4
4
same value
1
1
same value
6
6
same value
5
5
same value
9
9
same value
4
4
same value
7
2
8
8
same value
3
3
same value
2
7
4
4
same value
2
2
same value
7
7
same value
1
1
same value
8
8
same value
3
